<a href="https://colab.research.google.com/github/cotrader/Regime/blob/main/Multi_Regime_Agent_yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi‑Regime Agent (yfinance) – Bull/Bear, Volatility, Macro Proxy, Crypto

Dieses Notebook enthält modulare Python‑Funktionen (je Indikator/Regime eine Funktion) und einen **Multi‑Regime Agent**, der automatisch erkennt:

- **Bull / Bear Regime** (Trend)
- **Volatility Regime** (Realized Vol)
- **Macro Regime (Proxy)** (Risk‑On/Risk‑Off via Cross‑Asset Ratios)
- **Crypto‑specific Regime** (BTC/ETH Trend + Vol)
- daraus ein **Trading‑Signal** ableitet

> **Hinweis:** Das Macro‑Regime ist ein **Proxy** auf Basis von Marktdaten (SPY/TLT und HYG/IEF). Das ist robust ohne externe Makro‑Datenfeeds.


## 0) Installation (falls nötig)

Führe diese Zelle nur aus, wenn dir Pakete fehlen.


In [1]:
%pip -q install yfinance pandas numpy scikit-learn joblib

## 1) Imports

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from dataclasses import dataclass
from typing import Dict, Optional, Tuple


## 2) Funktionen (je Indikator/Regime eine Funktion)

In [11]:
from __future__ import annotations

# =========================
# 0) Data loader (yfinance)
# =========================

def load_price_data(
    ticker: str,
    start: str = "2015-01-01",
    interval: str = "1d",
    auto_adjust: bool = True,
) -> pd.DataFrame:
    """
    Loads OHLCV data via yfinance and returns a clean DataFrame.
    """
    df = yf.download(
        tickers=ticker,
        start=start,
        interval=interval,
        auto_adjust=auto_adjust,
        progress=False,
        group_by="column",
        threads=True,
    )

    if df is None or len(df) == 0:
        raise ValueError(f"No data returned for ticker: {ticker}")

    # Handle yfinance MultiIndex columns if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    if "Close" not in df.columns:
        raise ValueError(f"Missing Close column for {ticker}. Columns: {list(df.columns)}")

    df = df.dropna(subset=["Close"]).copy()
    df.index = pd.to_datetime(df.index)
    return df


def compute_returns(df: pd.DataFrame, price_col: str = "Close") -> pd.Series:
    """
    Log returns are more stable for vol calculations.
    """
    return np.log(df[price_col]).diff().dropna()


# ==================================
# 1) Bull/Bear Regime (Trend signal)
# ==================================

def indicator_ma_cross(
    df: pd.DataFrame,
    fast: int = 50,
    slow: int = 200,
    price_col: str = "Close",
) -> pd.DataFrame:
    """
    Adds MA fast/slow and MA-cross signal:
    +1 bullish (fast>slow), -1 bearish (fast<slow)
    """
    out = df.copy()
    out[f"MA{fast}"] = out[price_col].rolling(fast).mean()
    out[f"MA{slow}"] = out[price_col].rolling(slow).mean()
    out["ma_cross_signal"] = np.where(out[f"MA{fast}"] > out[f"MA{slow}"], 1, -1)
    return out


def regime_bull_bear(
    df: pd.DataFrame,
    fast: int = 50,
    slow: int = 200,
    confirm_price_vs_slow: bool = True,
    price_col: str = "Close",
) -> Dict:
    """
    Bull/Bear classification.
    If confirm_price_vs_slow=True, requires Close > MA(slow) for Bull.
    """
    tmp = indicator_ma_cross(df, fast=fast, slow=slow, price_col=price_col).dropna()
    if len(tmp) == 0:
        return {"regime": "Unknown", "signal": 0, "reason": "Not enough data for MAs"}

    last = tmp.iloc[-1]
    ma_fast = float(last[f"MA{fast}"])
    ma_slow = float(last[f"MA{slow}"])
    close = float(last[price_col])

    bull = (ma_fast > ma_slow)
    if confirm_price_vs_slow:
        bull = bull and (close > ma_slow)

    regime = "Bull" if bull else "Bear"
    signal = 1 if bull else -1

    return {
        "regime": regime,
        "signal": signal,
        "close": close,
        f"MA{fast}": ma_fast,
        f"MA{slow}": ma_slow,
        "confirm_price_vs_slow": confirm_price_vs_slow,
    }


# ======================================
# 2) Volatility Regime (Realized Vol)
# ======================================

def indicator_realized_vol(
    df: pd.DataFrame,
    window: int = 20,
    annualization: int = 252,
    price_col: str = "Close",
) -> pd.Series:
    """
    Realized volatility based on log returns.
    """
    r = compute_returns(df, price_col=price_col)
    vol = r.rolling(window).std() * np.sqrt(annualization)
    return vol


def regime_volatility(
    df: pd.DataFrame,
    window: int = 20,
    lookback_for_threshold: int = 252,
    annualization: int = 252,
    price_col: str = "Close",
) -> Dict:
    """
    Classifies volatility as Low / Medium / High using rolling thresholds
    from the last `lookback_for_threshold` observations:
      - Low:    vol <= 33rd percentile
      - Medium: between 33rd and 67th
      - High:   vol >= 67th percentile
    """
    vol = indicator_realized_vol(df, window=window, annualization=annualization, price_col=price_col).dropna()
    if len(vol) < max(60, window + 5):
        return {"regime": "Unknown", "signal": 0, "reason": "Not enough data for vol regime"}

    recent = vol.iloc[-lookback_for_threshold:] if len(vol) > lookback_for_threshold else vol
    q33 = float(np.nanpercentile(recent, 33))
    q67 = float(np.nanpercentile(recent, 67))
    v = float(vol.iloc[-1])

    if v <= q33:
        regime = "LowVol"
        signal = 1   # risk-on friendly
    elif v >= q67:
        regime = "HighVol"
        signal = -1  # risk-off
    else:
        regime = "MedVol"
        signal = 0

    return {
        "regime": regime,
        "signal": signal,
        "realized_vol": v,
        "q33": q33,
        "q67": q67,
        "window": window,
        "lookback_for_threshold": lookback_for_threshold,
    }


# ==========================================================
# 3) Macro Regime (Proxy) via Cross-Asset Risk-On / Risk-Off
# ==========================================================

def indicator_ratio_trend(
    numerator: pd.DataFrame,
    denominator: pd.DataFrame,
    fast: int = 50,
    slow: int = 200,
    price_col: str = "Close",
) -> Dict:
    """
    Computes ratio = num/den and returns MA cross signal on the ratio.
    """
    idx = numerator.index.intersection(denominator.index)
    if len(idx) < slow + 10:
        return {"signal": 0, "reason": "Not enough overlapping history"}

    ratio = (numerator.loc[idx, price_col] / denominator.loc[idx, price_col]).dropna()
    df_ratio = pd.DataFrame({"Close": ratio})
    tmp = indicator_ma_cross(df_ratio, fast=fast, slow=slow, price_col="Close").dropna()
    last = tmp.iloc[-1]
    return {
        "ratio_close": float(last["Close"]),
        f"ratio_MA{fast}": float(last[f"MA{fast}"]),
        f"ratio_MA{slow}": float(last[f"MA{slow}"]),
        "signal": int(last["ma_cross_signal"]),
    }


def regime_macro_proxy(
    start: str = "2010-01-01",
    fast: int = 50,
    slow: int = 200,
) -> Dict:
    """
    Macro regime proxy using:
      - SPY / TLT (equities vs duration)
      - HYG / IEF (credit risk vs safer bonds)

    If both are bullish => RiskOn
    If both bearish => RiskOff
    Else => Mixed
    """
    spy = load_price_data("SPY", start=start)
    tlt = load_price_data("TLT", start=start)
    hyg = load_price_data("HYG", start=start)
    ief = load_price_data("IEF", start=start)

    r1 = indicator_ratio_trend(spy, tlt, fast=fast, slow=slow)
    r2 = indicator_ratio_trend(hyg, ief, fast=fast, slow=slow)

    s1 = r1.get("signal", 0)
    s2 = r2.get("signal", 0)

    if s1 == 1 and s2 == 1:
        regime = "RiskOn"
        signal = 1
    elif s1 == -1 and s2 == -1:
        regime = "RiskOff"
        signal = -1
    else:
        regime = "Mixed"
        signal = 0

    return {
        "regime": regime,
        "signal": signal,
        "SPY_over_TLT": r1,
        "HYG_over_IEF": r2,
        "fast": fast,
        "slow": slow,
    }


# ==========================================
# 4) Crypto-specific Regime (BTC + Vol)
# ==========================================

def regime_crypto(
    crypto_ticker: str = "BTC-USD",
    start: str = "2015-01-01",
    trend_fast: int = 50,
    trend_slow: int = 200,
    vol_window: int = 20,
) -> Dict:
    """
    Crypto regime using:
      - Trend regime (Bull/Bear) on BTC (or chosen crypto)
      - Vol regime on BTC
    Combines to:
      - Bull + Low/MedVol => "CryptoRiskOn"
      - Bear or HighVol   => "CryptoRiskOff"
      - else Mixed
    """
    df = load_price_data(crypto_ticker, start=start)

    trend = regime_bull_bear(df, fast=trend_fast, slow=trend_slow, confirm_price_vs_slow=True)
    # crypto trades ~365 days a year, so annualization=365 is a decent approximation
    vol = regime_volatility(df, window=vol_window, lookback_for_threshold=252, annualization=365)

    if trend["signal"] == 1 and vol["regime"] in ("LowVol", "MedVol"):
        regime = "CryptoRiskOn"
        signal = 1
    elif trend["signal"] == -1 or vol["regime"] == "HighVol":
        regime = "CryptoRiskOff"
        signal = -1
    else:
        regime = "CryptoMixed"
        signal = 0

    return {
        "regime": regime,
        "signal": signal,
        "ticker": crypto_ticker,
        "trend": trend,
        "vol": vol,
    }


# ==========================================
# 5) Multi-Regime Agent + Trading Signals
# ==========================================

@dataclass
class MultiRegimeConfig:
    primary_ticker: str = "SPY"
    crypto_ticker: str = "BTC-USD"
    start: str = "2015-01-01"
    trend_fast: int = 50
    trend_slow: int = 200
    vol_window: int = 20
    macro_start: str = "2010-01-01"


def generate_trading_signal(regimes: Dict) -> Dict:
    """
    Conservative rule-based signal engine.
    Output:
      - action: "RISK_ON", "RISK_OFF", "NEUTRAL"
      - position_sizing_hint: 0..1
    """
    bull_signal = regimes["bull_bear"]["signal"]
    vol_signal = regimes["volatility"]["signal"]
    macro_signal = regimes["macro"]["signal"]
    crypto_signal = regimes["crypto"]["signal"]

    score = bull_signal + vol_signal + macro_signal + crypto_signal  # approx -4..+4

    if score >= 2:
        action = "RISK_ON"
        size = min(1.0, 0.5 + 0.15 * score)
    elif score <= -2:
        action = "RISK_OFF"
        size = min(1.0, 0.5 + 0.15 * abs(score))
    else:
        action = "NEUTRAL"
        size = 0.35

    return {
        "score": int(score),
        "action": action,
        "position_sizing_hint": float(round(size, 2)),
        "explanation": {
            "bull_bear": regimes["bull_bear"]["regime"],
            "volatility": regimes["volatility"]["regime"],
            "macro": regimes["macro"]["regime"],
            "crypto": regimes["crypto"]["regime"],
        }
    }


def multi_regime_agent(cfg: MultiRegimeConfig) -> Dict:
    """
    Runs all regime detectors and returns a single structured object.
    """
    df_primary = load_price_data(cfg.primary_ticker, start=cfg.start)

    bull_bear = regime_bull_bear(
        df_primary,
        fast=cfg.trend_fast,
        slow=cfg.trend_slow,
        confirm_price_vs_slow=True,
    )

    vol = regime_volatility(
        df_primary,
        window=cfg.vol_window,
        lookback_for_threshold=252,
        annualization=252,
    )

    macro = regime_macro_proxy(
        start=cfg.macro_start,
        fast=cfg.trend_fast,
        slow=cfg.trend_slow,
    )

    crypto = regime_crypto(
        crypto_ticker=cfg.crypto_ticker,
        start=cfg.start,
        trend_fast=cfg.trend_fast,
        trend_slow=cfg.trend_slow,
        vol_window=cfg.vol_window,
    )

    regimes = {
        "asof": str(df_primary.index[-1].date()),
        "primary_ticker": cfg.primary_ticker,
        "bull_bear": bull_bear,
        "volatility": vol,
        "macro": macro,
        "crypto": crypto,
    }

    signal = generate_trading_signal(regimes)

    return {
        "regimes": regimes,
        "trading_signal": signal,
    }

## 3) Quick Run (Beispiel)

Passe `primary_ticker` und `crypto_ticker` an deine Assets an, z. B.:

- `primary_ticker="QQQ"` für Nasdaq
- `crypto_ticker="ETH-USD"` für Ethereum


In [12]:
# Fix für yfinance MultiIndex-Problem bei einzelnen Tickern
import yfinance as yf
try:
    yf.set_price_granular_patch(True)
except AttributeError:
    # Falls die Version den Patch nicht kennt, erzwingen wir das alte Verhalten via Download-Parameter
    pass

cfg = MultiRegimeConfig(primary_ticker="SPY", crypto_ticker="BTC-USD", start="2015-01-01")
out = multi_regime_agent(cfg)

out["regimes"]["asof"], out["trading_signal"]

('2026-02-13',
 {'score': 1,
  'action': 'NEUTRAL',
  'position_sizing_hint': 0.35,
  'explanation': {'bull_bear': 'Bull',
   'volatility': 'MedVol',
   'macro': 'RiskOn',
   'crypto': 'CryptoRiskOff'}})

## 4) Ergebnis hübsch anzeigen

In [13]:
import json
print("As of:", out["regimes"]["asof"])
print("\nTrading Signal:")
print(json.dumps(out["trading_signal"], indent=2))
print("\nRegimes (short):")
print(json.dumps(out["regimes"], indent=2)[:1500] + " ...")

As of: 2026-02-13

Trading Signal:
{
  "score": 1,
  "action": "NEUTRAL",
  "position_sizing_hint": 0.35,
  "explanation": {
    "bull_bear": "Bull",
    "volatility": "MedVol",
    "macro": "RiskOn",
    "crypto": "CryptoRiskOff"
  }
}

Regimes (short):
{
  "asof": "2026-02-13",
  "primary_ticker": "SPY",
  "bull_bear": {
    "regime": "Bull",
    "signal": 1,
    "close": 681.75,
    "MA50": 686.8730090332032,
    "MA200": 645.9951885986328,
    "confirm_price_vs_slow": true
  },
  "volatility": {
    "regime": "MedVol",
    "signal": 0,
    "realized_vol": 0.1428121060340816,
    "q33": 0.1041184889596396,
    "q67": 0.14327499812271932,
    "window": 20,
    "lookback_for_threshold": 252
  },
  "macro": {
    "regime": "RiskOn",
    "signal": 1,
    "SPY_over_TLT": {
      "ratio_close": 7.598640110614314,
      "ratio_MA50": 7.856822017171146,
      "ratio_MA200": 7.474266131789614,
      "signal": 1
    },
    "HYG_over_IEF": {
      "ratio_close": 0.8317045492806229,
      "rati

## 5) Nächste Schritte (optional)

- **Backtest:** Score/Signal vs. Buy&Hold
- **Streamlit Dashboard:** Kacheln für Regime + Score + Position‑Sizing
- **Execution Adapter:** nur Paper Trading oder API‑Orders (z. B. Binance/Bybit)

Wenn du willst, kann ich dir die nächste Version direkt als:
1) Backtest‑Notebook oder  
2) Streamlit‑App (Dashboard)  
bereitstellen.


# Task
Extract the trading signals and regime results from the `out` variable, structure them into a flat pandas DataFrame, and export this data to an Excel file named "trading_signals.xlsx". Provide a summary of the exported data and a link to download the file from the notebook environment.

## Prepare Data for Export

### Subtask:
Extract the current regime results and trading signals from the 'out' variable and structure them into a flat pandas DataFrame suitable for Excel export.


**Reasoning**:
I will create a python code block to flatten the nested 'out' dictionary into a single-level structure and then convert it into a pandas DataFrame for easier export.



In [14]:
import pandas as pd

# Extract core components
regimes = out['regimes']
signal = out['trading_signal']

# Flatten the nested structure into a single dictionary
flat_data = {
    'asof': regimes['asof'],
    'primary_ticker': regimes['primary_ticker'],
    'trading_score': signal['score'],
    'trading_action': signal['action'],
    'position_sizing_hint': signal['position_sizing_hint'],
    # Bull/Bear Details
    'regime_bull_bear': regimes['bull_bear']['regime'],
    'signal_bull_bear': regimes['bull_bear']['signal'],
    'close_primary': regimes['bull_bear'].get('close'),
    # Volatility Details
    'regime_volatility': regimes['volatility']['regime'],
    'realized_vol': regimes['volatility'].get('realized_vol'),
    # Macro Details
    'regime_macro': regimes['macro']['regime'],
    'macro_signal': regimes['macro']['signal'],
    # Crypto Details
    'crypto_ticker': regimes['crypto']['ticker'],
    'regime_crypto': regimes['crypto']['regime'],
    'crypto_signal': regimes['crypto']['signal'],
    'crypto_vol': regimes['crypto']['vol'].get('realized_vol')
}

# Create DataFrame
df_export = pd.DataFrame([flat_data])

# Display the resulting DataFrame structure
print("DataFrame for export created successfully.")
print("Columns:", df_export.columns.tolist())
df_export.head()

DataFrame for export created successfully.
Columns: ['asof', 'primary_ticker', 'trading_score', 'trading_action', 'position_sizing_hint', 'regime_bull_bear', 'signal_bull_bear', 'close_primary', 'regime_volatility', 'realized_vol', 'regime_macro', 'macro_signal', 'crypto_ticker', 'regime_crypto', 'crypto_signal', 'crypto_vol']


,asof,primary_ticker,trading_score,trading_action,position_sizing_hint,regime_bull_bear,signal_bull_bear,close_primary,regime_volatility,realized_vol,regime_macro,macro_signal,crypto_ticker,regime_crypto,crypto_signal,crypto_vol
0,2026-02-13,SPY,1,NEUTRAL,0.35,Bull,1,681.75,MedVol,0.142812,RiskOn,1,BTC-USD,CryptoRiskOff,-1,0.978887


## Export to Excel

### Subtask:
Save the DataFrame as an Excel file named 'trading_signals.xlsx' using pandas and provide a link or instructions to download it from the notebook environment.


**Reasoning**:
I will export the df_export DataFrame to an Excel file and use the google.colab library to trigger an automatic download for the user.



In [15]:
try:
    import google.colab.files as colab_files
    # Export to Excel
    excel_filename = 'trading_signals.xlsx'
    df_export.to_excel(excel_filename, index=False)

    # Trigger download
    colab_files.download(excel_filename)
    print(f"Successfully saved data to {excel_filename} and initiated download.")
except ImportError:
    # Fallback if not in a Colab environment
    df_export.to_excel('trading_signals.xlsx', index=False)
    print("File 'trading_signals.xlsx' has been saved to the local directory. (Colab download tool not available in this environment)")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully saved data to trading_signals.xlsx and initiated download.


## Final Task

### Subtask:
Confirm the successful creation of the Excel file and summarize the exported data.


## Summary:

### Q&A

**What data was exported and what is the current trading status?**
The exported data includes a multi-regime analysis for the primary ticker SPY as of 2026-02-13. The current analysis shows a "Bull" regime for SPY with a trading score of 1, resulting in a "NEUTRAL" trading action. It also incorporates crypto data (BTC-USD), which currently indicates a "CryptoRiskOff" signal.

**How can the results be accessed outside the notebook?**
The results have been structured into a flat DataFrame and exported to an Excel file named `trading_signals.xlsx`, which was automatically prompted for download via the browser.

### Data Analysis Key Findings

*   **Data Consolidation:** A nested dictionary structure was successfully flattened into a single-row DataFrame with 16 distinct columns, merging technical, macro, volatility, and crypto-specific indicators.
*   **Regime Metrics:** The primary ticker (SPY) is currently in a Bull regime, though the overall trading action is neutral.
*   **Volatility and Macro Indicators:** The data capture includes realized volatility for both equities and crypto, alongside macro regime signals, providing a holistic view of the market environment.
*   **Export Integrity:** The final Excel file, `trading_signals.xlsx`, contains the complete snapshot of the `asof` date (2026-02-13), ensuring data traceability for future audits.

### Insights or Next Steps

*   **Automated Monitoring:** Since the export structure is now standardized, this process can be scheduled to generate daily reports to track how the "NEUTRAL" trading action evolves as volatility or macro signals shift.
*   **Expansion of Tickers:** The flattened data structure is designed for a single primary ticker; future analysis could iterate through a watchlist to create a multi-row Excel summary of signals across different asset classes.


# Task
Generalize the trading signal extraction logic to treat crypto and equity tickers identically within a unified regime framework. Flatten the resulting multi-asset indicators (Trend, Vol, Macro) into a standardized pandas DataFrame where each ticker is represented by a single row. Finally, export these unified results to an Excel file named "trading_signals.xlsx" and summarize the findings.

## Generalize Signal Extraction

### Subtask:
Refactor the data extraction and regime calculation logic to handle both equities and crypto using a single, unified function.


**Reasoning**:
I will define a unified function `get_asset_regime` to standardize the extraction process for both equity and crypto assets and then iterate through a list of tickers to collect their regime data.



In [16]:
def get_asset_regime(ticker: str, asset_type: str = "equity") -> dict:
    """
    Unified function to extract trend and volatility regimes for any asset.
    """
    # 365 days for crypto, 252 for others
    ann_factor = 365 if "USD" in ticker or asset_type == "crypto" else 252

    # Load data
    df = load_price_data(ticker, start="2015-01-01")

    # Compute Bull/Bear (Trend)
    trend = regime_bull_bear(df, fast=50, slow=200)

    # Compute Volatility
    vol = regime_volatility(df, window=20, annualization=ann_factor)

    # Standardize output structure
    return {
        "ticker": ticker,
        "asset_type": asset_type,
        "asof": str(df.index[-1].date()),
        "regime_trend": trend["regime"],
        "trend_signal": trend["signal"],
        "regime_vol": vol["regime"],
        "vol_signal": vol["signal"],
        "realized_vol": vol.get("realized_vol"),
        "close_price": float(df["Close"].iloc[-1])
    }

# Define tickers and process them
tickers_to_analyze = [("SPY", "equity"), ("QQQ", "equity"), ("BTC-USD", "crypto"), ("ETH-USD", "crypto")]
unified_regime_results = []

for ticker, a_type in tickers_to_analyze:
    try:
        res = get_asset_regime(ticker, asset_type=a_type)
        unified_regime_results.append(res)
        print(f"Processed {ticker} successfully.")
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Preview the collected data
import pandas as pd
df_unified_preview = pd.DataFrame(unified_regime_results)
df_unified_preview.head()

Processed SPY successfully.
Processed QQQ successfully.
Processed BTC-USD successfully.
Processed ETH-USD successfully.


,ticker,asset_type,asof,regime_trend,trend_signal,regime_vol,vol_signal,realized_vol,close_price
0,SPY,equity,2026-02-13,Bull,1,MedVol,0,0.142812,681.750000
1,QQQ,equity,2026-02-13,Bull,1,MedVol,0,0.189390,601.919983
2,BTC-USD,crypto,2026-02-14,Bear,-1,HighVol,-1,0.978525,69708.382812
3,ETH-USD,crypto,2026-02-14,Bear,-1,HighVol,-1,1.184953,2081.377441


## Standardize DataFrame for Export

### Subtask:
Flatten the regime results into a standardized pandas DataFrame where each row represents a ticker and its associated indicators (Trend, Vol, Macro).


**Reasoning**:
I will create a unified DataFrame from the collected regime results, augment it with current macro indicators by calling the regime_macro_proxy function, and organize the columns for a standardized export format.



In [17]:
import pandas as pd

# 1. Create DataFrame from unified_regime_results
df_unified_final = pd.DataFrame(unified_regime_results)

# 2. Add current macro regime status to each row
macro_data = regime_macro_proxy()

df_unified_final['regime_macro'] = macro_data['regime']
df_unified_final['macro_signal'] = macro_data['signal']

# 3. Reorder columns logically
column_order = [
    'ticker', 'asset_type', 'asof',
    'regime_trend', 'trend_signal', 'close_price',
    'regime_vol', 'vol_signal', 'realized_vol',
    'regime_macro', 'macro_signal'
]

df_unified_final = df_unified_final[column_order]

# 5. Display the final DataFrame
print("Final unified DataFrame (df_unified_final) created successfully.")
df_unified_final.head()

Final unified DataFrame (df_unified_final) created successfully.


,ticker,asset_type,asof,regime_trend,trend_signal,close_price,regime_vol,vol_signal,realized_vol,regime_macro,macro_signal
0,SPY,equity,2026-02-13,Bull,1,681.750000,MedVol,0,0.142812,RiskOn,1
1,QQQ,equity,2026-02-13,Bull,1,601.919983,MedVol,0,0.189390,RiskOn,1
2,BTC-USD,crypto,2026-02-14,Bear,-1,69708.382812,HighVol,-1,0.978525,RiskOn,1
3,ETH-USD,crypto,2026-02-14,Bear,-1,2081.377441,HighVol,-1,1.184953,RiskOn,1


## Export Unified Results to Excel

### Subtask:
Save the standardized DataFrame to 'trading_signals.xlsx' and trigger the download for the user.


**Reasoning**:
I will export the unified results stored in df_unified_final to an Excel file and use the google.colab files module to trigger an automatic download as requested.



In [18]:
from google.colab import files

# Export to Excel
excel_filename = 'trading_signals.xlsx'
df_unified_final.to_excel(excel_filename, index=False)

# Trigger download
files.download(excel_filename)
print(f"Confirmation: '{excel_filename}' has been created and the download has been triggered successfully.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Confirmation: 'trading_signals.xlsx' has been created and the download has been triggered successfully.


## Final Task

### Subtask:
Summarize the exported data and confirm that all assets are now treated under a unified regime framework.


## Summary:

### Q&A

**How are crypto and equity assets treated under the unified regime framework?**
Both asset classes are processed through a single function, `get_asset_regime`, which standardizes the logic for trend and volatility. The main distinction is the annualization factor, which is dynamically set to 365 days for cryptocurrencies and 252 days for equities.

**What is the current market status across the analyzed assets?**
The analysis shows a divergence: equities (SPY, QQQ) are currently in a **Bull** trend with **Medium Volatility**, while cryptocurrencies (BTC, ETH) are in a **Bear** trend with **High Volatility**. All assets share a common **RiskOn** macro regime.

### Data Analysis Key Findings
*   **Unified Indicators**: A standardized dataset was generated for SPY, QQQ, BTC-USD, and ETH-USD, including trend signals, volatility regimes, and macro status.
*   **Diverging Trends**: Equities maintained a positive trend signal (1), whereas crypto assets showed a negative trend signal (-1).
*   **Volatility Profiles**: Realized volatility for cryptocurrencies was significantly higher than for equities, resulting in a **HighVol** classification (signal -1) for crypto versus **MedVol** (signal 0) for equities.
*   **Macro Synchronization**: All assets were tagged with a global "RiskOn" macro regime (signal 1) using the `regime_macro_proxy` logic.
*   **Export and Delivery**: The final data was flattened into a single-row-per-ticker format and successfully exported to an Excel file named `trading_signals.xlsx`.

### Insights or Next Steps
*   **Portfolio Rebalancing**: Given the bear trend and high volatility in crypto compared to the bull trend in equities, a rotation or hedging strategy might be considered despite the "RiskOn" macro signal.
*   **Next Step**: Integrate specific momentum indicators or relative strength index (RSI) metrics into the unified framework to provide more granular entry/exit signals alongside the regime classifications.
